In [108]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [109]:
import pandas as pd
import numpy as np
from datetime import datetime

# Raw data

In [110]:
data = pd.read_csv('../../raw_data/210908_LeWagon_finalproject_v3.csv')
data.head(2)

,SDC Deal No,Date Announced,"Deal Value\r\n(USD, Millions)","Acquiror Total Assets Last 12 Months\r\n(USD, Millions)",Acquiror Full Name,Acquiror Primary Ticker Symbol,Target Full Name,Target Nation,Acquiror Nation,Target Public Status,Acquiror TRBC Industry,Target TRBC Industry,Percentage of Shares Held at Announcement,Percentage of Shares Acquired in Transaction,Consideration Offered\r\n('|'),Deal Attitude,Acquisition Techniques\r\n('|'),Acquiror Financial Advisors Name\r\n('|'),Target Financial Advisors Name\r\n('|'),Purpose\r\n('|')
0,1631732020,01/01/2005,12.00,"3,160.41",Plains All American Pipeline LP,PAA,Shell Pipeline Co LP-Crude Oil Pipeline Assets...,United States,United States,Subsidiary,Oil & Gas Refining and Marketing,Oil & Gas Transportation Services,NaN,100.0,Cash|Cash Only,Friendly,Financial Acquiror|Divestiture,NaN,NaN,Strengthen existing operations/expand presence...
1,1653652020,01/01/2005,NaN,"1,680.57",Regis Corp,RGS,Scot Lewis Schools,United States,United States,Private,Personal Services,Professional & Business Education,NaN,100.0,Unspecified,Friendly,Not Applicable,NaN,NaN,NaN


# Adjust data

## Column names

In [111]:
data.columns

Index(['SDC Deal No', 'Date Announced', 'Deal Value\r\n(USD, Millions)',
       'Acquiror Total Assets Last 12 Months\r\n(USD, Millions)',
       'Acquiror Full Name', 'Acquiror Primary Ticker Symbol',
       'Target Full Name', 'Target Nation', 'Acquiror Nation',
       'Target Public Status', 'Acquiror TRBC Industry',
       'Target TRBC Industry', 'Percentage of Shares Held at Announcement',
       'Percentage of Shares Acquired in Transaction',
       'Consideration Offered\r\n('|')', 'Deal Attitude',
       'Acquisition Techniques\r\n('|')',
       'Acquiror Financial Advisors Name\r\n('|')',
       'Target Financial Advisors Name\r\n('|')', 'Purpose\r\n('|')'],
      dtype='object')

In [112]:
new_columns = [
    'id', 'announcement_date', 'deal_value', 'acquiror_total_assets', 'acquiror_name', 'acquiror_ticker', 'target_name',
    'target_nation', 'acquiror_nation', 'target_status', 'acquiror_industry', 'target_industry',
    'shares_at_announcement', 'shares_acquired', 'consideration_offered', 'attitude',
    'acquisition_technique', 'acquiror_financial_advisor', 'target_financial_advisor', 'purpose'
]

In [113]:
data.columns = new_columns

## Adding TRBC (industry classifier)

In [114]:
trbc = pd.read_pickle('../../MA_PREDICTOR/data/trbc.pkl')

,Title,Hierarchical_Code
0,Coal,50101010
1,Integrated Oil & Gas,50102010
2,Oil & Gas Exploration and Production,50102020
3,Oil & Gas Refining and Marketing,50102030
4,Oil & Gas Drilling,50103010


In [115]:
# Acquiror
data = data.join(trbc.set_index('Title'), on='acquiror_industry')
data.rename(columns={'Hierarchical_Code': 'acquiror_code'}, inplace=True)

# Target
data = data.join(trbc.set_index('Title'), on='target_industry')
data.rename(columns={'Hierarchical_Code': 'target_code'}, inplace=True)

## Adding acquisition_count

In [116]:
# Loading acquisition data from 1990-2021
hist_acquisition = pd.read_pickle('../../MA_PREDICTOR/data/acquisition_count.pkl')
hist_acquisition.drop(columns=['announcement_date'], inplace=True)

In [117]:
data = data.join(hist_acquisition.set_index('id')['acquisition_count'], on='id')

In [118]:
# We have to fill Nan's with previous value for cumcount for this specific acuqiror

# Get all acquirors' name in dataset
acquirors = set(data.acquiror_name.unique())

# Get all acquirors' name in historical dataset
hist_acquirors = set(hist_acquisition.acquiror_name.unique())

# Iterate through set of acquirors and fill NaN values accordingly
for acquiror in acquirors:
    
    # Set deals per acquiror as variable
    series = data.acquisition_count[data.acquiror_name == acquiror]
    
    # Special fill if first deal in dataset is not completed (ffill not suited)
    if pd.isna(series.iloc[0]):
        
        # No completed acquisition can be observed from 2005-2021 dataset
        if all(pd.isna(data.acquisition_count[data.acquiror_name == acquiror])):
            
            # Take most recent value (happening before 2005) and manually accumulate
            if acquiror in hist_acquirors:
                series.iloc[0] = hist_acquisition[hist_acquisition.acquiror_name == acquiror
                                                 ].acquisition_count.max() + 1

            # If not available in historical dataset, completed mergers are assumed to be 0
            else:
                series.iloc[0] = 0
                
        # Find next available number of completed acquisition and manually decumulate
        else:
            for i in range(len(series)):
                if not pd.isna(series.iloc[i]):
                    series.iloc[0] = series.iloc[i] - 1
                    break
                    
    # With first observation filled we can forwardfill the rest
    series = series.ffill()
                
    data.acquisition_count[data.acquiror_name == acquiror] = series

/var/folders/z8/077xt3wd30j28k4_3vtz7gn00000gn/T/ipykernel_28523/3583381704.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.acquisition_count[data.acquiror_name == acquiror] = series


## Adding bidder_count

In [119]:
bidder = pd.read_csv('../../raw_data/210914_bidder_count.csv')

,SDC Deal No,Date Announced,"Deal Value\n(USD, Millions)","Acquiror Total Assets Last 12 Months\n(USD, Millions)",Acquiror Full Name,Acquiror Primary Ticker Symbol,Target Full Name,Target Nation,Acquiror Nation,Target Public Status,...,Target TRBC Industry,Percentage of Shares Held at Announcement,Percentage of Shares Acquired in Transaction,Consideration Offered\n('|'),Deal Attitude,Acquisition Techniques\n('|'),Acquiror Financial Advisors Name\n('|'),Target Financial Advisors Name\n('|'),Purpose\n('|'),Number of Bidders
0,1631732020,01/01/2005,12.00,"3,160.41",Plains All American Pipeline LP,PAA,Shell Pipeline Co LP-Crude Oil Pipeline Assets...,United States,United States,Subsidiary,...,Oil & Gas Transportation Services,NaN,100.0,Cash Only|Cash,Friendly,Financial Acquiror|Divestiture,NaN,NaN,Strengthen existing operations/expand presence...,1.0
1,1653652020,01/01/2005,NaN,"1,680.57",Regis Corp,RGS,Scot Lewis Schools,United States,United States,Private,...,Professional & Business Education,NaN,100.0,Unspecified,Friendly,Not Applicable,NaN,NaN,NaN,1.0


In [120]:
bidder = bidder[['SDC Deal No', 'Number of Bidders']]
bidder.columns = ['id', 'bidder_count']

In [121]:
data = data.join(bidder.set_index('id'), on='id')

In [128]:
data['bidder_count'].fillna(1, inplace=True)

## dtypes

In [129]:
# transforming announcement_date
data['announcement_date'] = pd.to_datetime(data['announcement_date'], format="%d/%m/%Y")

In [130]:
# transforming deal_value
rem_com = lambda x: x.replace(',', '') if type(x) == str else x # removing comma in string, leaving float
num_feat = ['deal_value', 'acquiror_total_assets']
for feat in num_feat:
    data[feat]= data[feat].apply(rem_com).astype(float)

In [131]:
# Transforming trbc codes

# Removing nas
data = data[data['acquiror_code'].notna()]
data = data[data['target_code'].notna()]

# Type transformation
data['acquiror_code'] = data['acquiror_code'].astype(int)
data['target_code'] = data['target_code'].astype(int)

In [132]:
data.isna().sum()

id                                0
announcement_date                 0
deal_value                    17813
acquiror_total_assets          8738
acquiror_name                     0
acquiror_ticker                  73
target_name                       1
target_nation                     1
acquiror_nation                   0
target_status                     0
acquiror_industry                 0
target_industry                   0
shares_at_announcement        40009
shares_acquired                   0
consideration_offered             7
attitude                          0
acquisition_technique             3
acquiror_financial_advisor    35036
target_financial_advisor      32109
purpose                       19147
acquiror_code                     0
target_code                       0
acquisition_count                 0
bidder_count                      0
dtype: int64

In [133]:
# Transforming acquisiton and bidder counts
data['acquisition_count'] = data['acquisition_count'].astype(int)
data['bidder_count'] = data['bidder_count'].astype(int)

# Modifying data

In [134]:
# deal_value=not considered
# acquiror_ticker=remove
# target_name/target_nation/target_industry=remove
# target_status=remove
# shares_at_announcement=0
# consideration_offered=remove
# acquiror/target_financial_advisor=not considered

## Fill up missing values

In [135]:
# NA in 'shares_at_announcement' implies 0
data['shares_at_announcement'].fillna(0, inplace=True)

## Removing NAs

In [136]:
drop_na = ['acquiror_ticker', 'target_name', 'target_nation', 'target_status', 'target_industry',
           'consideration_offered']

In [137]:
for col in drop_na:
    data = data[data[col].notna()]

## Optional: clean consideration offered

In [138]:
# 'Unspecified' in consideration_offered has to be removed -> we would lose 38% of our data
len(data[data['consideration_offered'] == 'Unspecified']) / len(data)

0.3874583606876261

In [139]:
# to be excluded in detailed dataset
# data = data[data['consideration_offered'] != 'Unspecified']

## US only

In [140]:
# Raw data was filtered for nation of incorporation, some nations stayed in
data['acquiror_nation'].unique()

array(['United States', 'Canada', 'China (Mainland)', 'Russia',
       'Hong Kong', 'Philippines', 'Poland', 'Japan', 'Netherlands',
       'France', 'Malaysia', 'United Kingdom', 'Australia', 'Taiwan',
       'Sweden', 'Bermuda', 'Switzerland', 'Hungary', 'Singapore',
       'Israel', 'Peru', 'Argentina', 'Colombia', 'United Arab Emirates',
       'Thailand', 'Brazil', 'Cambodia', 'New Zealand', 'Panama',
       'South Korea', 'Germany', 'Ireland', 'Italy', 'Ecuador', 'Vietnam',
       'Czech Republic', 'Dominican Republic', 'Fiji', 'Kenya', 'Denmark',
       'Ukraine', 'Latvia', 'Malta', 'Jamaica', 'Puerto Rico', 'Greece',
       'Indonesia', 'U.S. Virgin Islands', 'India'], dtype=object)

In [141]:
data = data[data['acquiror_nation'] == 'United States']

## Empty acquisitions

In [142]:
data[data['shares_acquired'] <= 0.0].shape # we have 12579 empty acquisitions

(12579, 24)

In [143]:
data = data[data['shares_acquired'] > 0.0]

## Optional: No purpose given

In [144]:
data[data['purpose'] == 'Other'].shape # 673 undefined purpose
# + 12533 NaN
# = 13206 lost deals

(673, 24)

In [145]:
# to be excluded in detailed model
#data = data[data['purpose'].notna()]
#data = data[data['purpose'] != 'Other']

## Optional: No acquisition_technique

In [146]:
data[data['acquisition_technique'] == 'Not Applicable'].shape # 13940 not specified

(13940, 24)

In [147]:
# to be excluded in detailed dataset
#data = data[data['acquisition_technique'] != 'Not Applicable']

## Optional: missing financial data

In [148]:
fin_feat = ['deal_value', 'acquiror_total_assets']
#for feat in fin_feat:
#    data = data[data[feat].notna()]

# Upload 1

In [149]:
data.dtypes

id                                     int64
announcement_date             datetime64[ns]
deal_value                           float64
acquiror_total_assets                float64
acquiror_name                         object
acquiror_ticker                       object
target_name                           object
target_nation                         object
acquiror_nation                       object
target_status                         object
acquiror_industry                     object
target_industry                       object
shares_at_announcement               float64
shares_acquired                      float64
consideration_offered                 object
attitude                              object
acquisition_technique                 object
acquiror_financial_advisor            object
target_financial_advisor              object
purpose                               object
acquiror_code                          int64
target_code                            int64
acquisitio

In [150]:
data.shape # detailed: 5047 deals, not_detailed: 27272 deals

(27272, 24)

In [151]:
data.to_csv('../../MA_PREDICTOR/data/ma_data.csv', index=False, date_format="%d/%m/%Y")

# Clean missing feature (CAR)

In [152]:
data_car = pd.read_csv('../../MA_PREDICTOR/data/ma_data_car.csv', parse_dates=['announcement_date'])

In [154]:
data_car.isna().sum() # we will loose around 10858 observations (before: 8813) and 2022 for detailed

id                                0
announcement_date                 0
deal_value                    12921
acquiror_total_assets          1882
acquiror_name                     0
acquiror_ticker                   0
target_name                       0
target_nation                     0
acquiror_nation                   0
target_status                     0
acquiror_industry                 0
target_industry                   0
shares_at_announcement            0
shares_acquired                   0
consideration_offered             0
attitude                          0
acquisition_technique             0
acquiror_financial_advisor    22135
target_financial_advisor      19527
purpose                       12533
acquiror_code                     0
target_code                       0
acquisition_count                 0
bidder_count                      0
car_1                         10838
car_3                         10842
car_5                         10845
car_10                      

In [155]:
# Clean na values of missing stock/ CAR data
cars = [1, 3, 5, 10]
for car in cars:
    data_car = data_car[data_car[f'car_{car}'].notna()]

# Upload 2

In [156]:
data_car.to_csv('../../MA_PREDICTOR/data/ma_data_car.csv', index=False, date_format="%d/%m/%Y")